In [1]:
import geopandas as gpd
import pandas as pd
from copy import deepcopy
from geopandas import GeoDataFrame
from shapely.ops import unary_union

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
gdf_center = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_center.shp')

In [5]:
#岩手県の線路データをダウンロード
gdf_width = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/src/数値地図/道路/3.shp', encoding='shift-jis') #このデータは道路中心線

In [6]:
#座標系を変更
src_proj = 4326 # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定

# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_center.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_center = gdf_center.to_crs(epsg=dst_proj)  # 変換後座標に変換

gdf_width.crs = f'epsg:{src_proj}'
gdf_width = gdf_width.to_crs(epsg=dst_proj)

In [7]:
#gdf_widthを高速道路のみに絞る
gdf_width_highway = gdf_width[gdf_width['rdCtg'] == '高速自動車国道等']

# #gdf_osyu_width_highwayのindexを並び替える
gdf_width_highway.reset_index(drop=True, inplace=True)

In [8]:
#高速道路のデータフレームの頭にhighwayをつける

#すべてのカラムを取得
highway_columns = gdf_width_highway.columns.to_list()

#ループで各カラムを取得
for i in highway_columns[:22]:
  new_column = 'highway_' + i
  gdf_width_highway.rename(columns={f'{i}': f'{new_column}'}, inplace=True)

<ipython-input-8-7d84906274ee>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_width_highway.rename(columns={f'{i}': f'{new_column}'}, inplace=True)
<ipython-input-8-7d84906274ee>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_width_highway.rename(columns={f'{i}': f'{new_column}'}, inplace=True)
<ipython-input-8-7d84906274ee>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_width_highway.rename(columns={f'{i}': f'{new_co

In [9]:
#空間結合をする
result = gpd.sjoin_nearest(gdf_center, gdf_width_highway, how="left",lsuffix='point',rsuffix='road')
#空間結合された道路のgeometryを追加
result = pd.merge(result, gdf_width_highway[['highway_rID', 'geometry']], on='highway_rID')
#geometryの名前を変更
result.rename(columns={'geometry_y':'nearest_highway'}, inplace=True)
#メモリの解放
del gdf_width_highway

In [10]:
#dictance_highwayカラムを追加
result['distance_highway'] = None

for i in range((len(result) // 50)+1):
    try:
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_highway'][50*i:50*(i+1)]))

        # ポイントとマルチラインストリングとの最短距離を計算
        result['distance_highway'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
        #一回ごとにメモリを解放
        del merged_lines
    except Exception as e:
        # エラーが発生した場合の処理
        print(f"Error occurred at the last batch: {e}")
        # ラインデータをマルチラインストリングに結合
        merged_lines = unary_union(list(result['nearest_highway'][50*(i+1):]))

        result['distance_highway'][50*(i+1):] = result.geometry_x[50*(i+1):].apply(lambda p: p.distance(merged_lines))

ストリーミング出力は最後の 5000 行に切り捨てられました。
<ipython-input-10-b3e4fcfea545>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_highway'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-10-b3e4fcfea545>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['distance_highway'][50*i:50*(i+1)] = result.geometry_x[50*i:50*(i+1)].apply(lambda p: p.distance(merged_lines))
<ipython-input-10-b3e4fcfea545>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [11]:
#重複を削除
result = result[~result.Meshcode.duplicated()]

#indexを並び替え
result.reset_index(inplace=True)

In [ ]:
result_columns = result.columns.to_list()[6:]
result_columns[:0] = ['Meshcode']
result[result_columns].to_csv('/content/drive/Shareddrives/geomap/周辺環境データ構築/dev/奥州市/奥州市　最近傍購高速道路/奥州市_最近傍高速道路.csv', encoding='shift-jis', index=False)